In [1]:
import bw2data as bd
from datetime import datetime

bd.projects.set_current("optimex_demo")
bd.databases

Databases dictionary with 8 object(s):
	biosphere3
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei3.10-SSP2-RCP19-2020
	ei3.10-SSP2-RCP19-2030
	ei3.10-SSP2-RCP19-2040
	ei3.10-SSP2-RCP19-2050
	foreground

In [2]:
db_bio = bd.Database("biosphere3") 

# Create a dict of databases keyed by year
dbs = {
    2020: bd.Database("ei3.10-SSP2-RCP19-2020"),
    2030: bd.Database("ei3.10-SSP2-RCP19-2030"),
    2040: bd.Database("ei3.10-SSP2-RCP19-2040"),
    2050: bd.Database("ei3.10-SSP2-RCP19-2050"),
}

# Add representative_time metadata for each database
for year, db in dbs.items():
    db.metadata["representative_time"] = datetime(year, 1, 1).isoformat()

eidb = bd.Database("ecoinvent-3.10-cutoff")
db_2020 =dbs[2020]

if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.write({})
foreground.register()

In [3]:
from collections import namedtuple
import numpy as np
import bw_temporalis as bwt

# === Define reusable structure for flow data ===
Flow = namedtuple("Flow", ["dataset", "amount", "type", "phase"])

# === Phase durations ===
timing_config = {
    "PEM_Electrolysis": {
        "construction": 2,
        "lifetime": 15,
        "decommissioning": 1,
    },
    "SMR": {
        "construction": 3,
        "lifetime": 25,
        "decommissioning": 2,
    }
}

# === Temporal distribution generators ===
def make_temporal_distributions(name):
    config = timing_config[name]
    c, l, d = config["construction"], config["lifetime"], config["decommissioning"]

    return {
        "pre_operation": lambda: bwt.easy_timedelta_distribution(
            start=0, end=c, resolution="Y", steps=c + 1,
            kind="triangular" if name == "SMR" else "uniform"
        ),
        "operation": lambda: bwt.easy_timedelta_distribution(
            start=c, end=c + l, resolution="Y", steps=l + 1, kind="uniform"
        ),
        "post_operation": lambda: bwt.TemporalDistribution(
            date=np.array([c + l + d], dtype="timedelta64[Y]"),
            amount=np.array([1])
        )
    }

temporal_distributions = {
    name: make_temporal_distributions(name)
    for name in timing_config
}

In [4]:

# === Important flows for PEM and SMR ===
important_flows_pem = {
    "h2_pem": Flow(db_2020.get(name="hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity", unit="kilogram"), 1.0, "production", "operation"),
    "electrolyzer_stack": Flow(db_2020.get(name="electrolyzer production, 1MWe, PEM, Stack"), 1.35e-6, "technosphere", "pre_operation"),
    "electrolyzer_bop": Flow(db_2020.get(name="electrolyzer production, 1MWe, PEM, Balance of Plant"), 3.37e-7, "technosphere", "pre_operation"),
    "electrolyzer_disposal_stack": Flow(db_2020.get(name="treatment of fuel cell stack, 1MWe, PEM"), -1.35e-6, "technosphere", "post_operation"),
    "electrolyzer_disposal_bop": Flow(db_2020.get(name="treatment of fuel cell balance of plant, 1MWe, PEM"), -3.37e-7, "technosphere", "post_operation"),
    "electricity": Flow(db_2020.get(name="market group for electricity, low voltage", location="WEU"), 54.0, "technosphere", "operation"),
    "water_deionised_europe": Flow(db_2020.get(name="market for water, deionised", location="Europe without Switzerland"), 14.0, "technosphere", "operation"),
    "oxygen_air": Flow(db_bio.get(name="Oxygen", categories=("air",)), 8.0, "biosphere", "operation"),
}

important_flows_smr = {
    "h2_smr": Flow(db_2020.get(name="hydrogen production, steam methane reforming", location="RER", unit="kilogram"), 1.0, "production", "operation"),
    "natural_gas_ch": Flow(db_2020.get(name="market for natural gas, high pressure", location="CH"), 0.0294, "technosphere", "operation"),
    "natural_gas_europe": Flow(db_2020.get(name="market group for natural gas, high pressure", location="Europe without Switzerland"), 4.22, "technosphere", "operation"),
    "chemical_factory_construction": Flow(db_2020.get(name="chemical factory construction, organics", location="RER"), 4.99e-10, "technosphere", "pre_operation"),
    "water_deionised_ch": Flow(db_2020.get(name="market for water, deionised", location="CH"), 0.0148, "technosphere", "operation"),
    "water_deionised_europe": Flow(db_2020.get(name="market for water, deionised", location="Europe without Switzerland"), 4.12, "technosphere", "operation"),
    "oxygen_air": Flow(db_bio.get(name="Oxygen", categories=("natural resource", "in air")), 6.07, "biosphere", "operation"),
    "cooling_water": Flow(db_bio.get(name="Water, cooling, unspecified natural origin", categories=("natural resource", "in water")), 0.355, "biosphere", "operation"),
    "co2_fossil": Flow(db_bio.get(name="Carbon dioxide, fossil", categories=("air",)), 8.35, "biosphere", "operation"),
}

In [5]:

# === Exchange generation function ===
def flow_dict_to_exchanges(flow_dict, process_type, activity_key):
    exchanges = []
    td = temporal_distributions[process_type]

    for _, flow in flow_dict.items():
        exchange = {
            "amount": flow.amount,
            "type": flow.type,
            "input": activity_key if flow.type == "production" else flow.dataset.key,
            "unit": flow.dataset["unit"],
            "temporal_distribution": td[flow.phase](),
            "operation": flow.phase == "operation",
        }
        exchanges.append(exchange)

    return exchanges

# === Define foreground activities ===
pem_key = ("foreground", "hydrogen_production_pem")
smr_key = ("foreground", "hydrogen_production_smr")

pem_activity = {
    pem_key: {
        "name": "Hydrogen via PEM electrolysis",
        "unit": "kilogram",
        "location": "RER",
        "reference product": "hydrogen",
        "exchanges": flow_dict_to_exchanges(important_flows_pem, "PEM_Electrolysis", pem_key),
        "operation_time_limits": (2,2+15),
    }
}

smr_activity = {
    smr_key: {
        "name": "Hydrogen via Steam Methane Reforming",
        "unit": "kilogram",
        "location": "RER",
        "reference product": "hydrogen",
        "exchanges": flow_dict_to_exchanges(important_flows_smr, "SMR", smr_key),
        "operation_time_limits": (3,3+25),
    }
}

# === Write to foreground database ===
foreground.write({**pem_activity, **smr_activity})
foreground.register()


15:42:01+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 1980.78it/s]


15:42:01+0200 [info     ] Vacuuming database            


In [6]:
flow_dict_to_exchanges(important_flows_pem, "PEM_Electrolysis", pem_key)

[{'amount': 1.0,
  'type': 'production',
  'input': ('foreground', 'hydrogen_production_pem'),
  'unit': 'kilogram',
  'temporal_distribution': TemporalDistribution instance with 16 values and total: 1,
  'operation': True},
 {'amount': 1.35e-06,
  'type': 'technosphere',
  'input': ('ei3.10-SSP2-RCP19-2020', 'eb5cba25a1af4211a18b6cd84a87c964'),
  'unit': 'unit',
  'temporal_distribution': TemporalDistribution instance with 3 values and total: 1,
  'operation': False},
 {'amount': 3.37e-07,
  'type': 'technosphere',
  'input': ('ei3.10-SSP2-RCP19-2020', 'a0d7a61264b94939b183ebd22c9538d0'),
  'unit': 'unit',
  'temporal_distribution': TemporalDistribution instance with 3 values and total: 1,
  'operation': False},
 {'amount': -1.35e-06,
  'type': 'technosphere',
  'input': ('ei3.10-SSP2-RCP19-2020', 'c5b2e439c2ce4534b251d0657da2c7da'),
  'unit': 'unit',
  'temporal_distribution': TemporalDistribution instance with 1 values and total: 1,
  'operation': False},
 {'amount': -3.37e-07,
  't

In [7]:
smr = foreground.get(name="Hydrogen via Steam Methane Reforming")
for exc in smr.exchanges():
    print(exc)

Exchange: 1.0 kilogram 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 0.0294 cubic meter 'market for natural gas, high pressure' (cubic meter, CH, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 4.22 cubic meter 'market group for natural gas, high pressure' (cubic meter, Europe without Switzerland, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 4.99e-10 unit 'chemical factory construction, organics' (unit, RER, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 0.0148 kilogram 'market for water, deionised' (kilogram, CH, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 4.12 kilogram 'market for water, deionised' (kilogram, Europe without Switzerland, None) to 'Hydrogen via Steam Methane Reforming' (kilogram, RER, None)>
Exchange: 6.07 kilogram 'Oxygen' (kilogram, None, ('

In [8]:
from datetime import datetime
method = method = ('ecoinvent-3.10', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')

# Define temporally distributed demand from 2025 to 2045
years = range(2025, 2045)
td_demand = bwt.TemporalDistribution(
    date=np.array([datetime(year, 1, 1).isoformat() for year in years], dtype='datetime64[s]'),
    amount=np.asarray([100] * len(years), dtype=float)  # 100 kg per year
)

In [9]:
from optimex import lca_processor

lca_config = lca_processor.LCAConfig(
    demand={"hydrogen": td_demand},
    temporal= {
        "start_date": datetime(2020, 1, 1),
        "temporal_resolution": "year",
        "time_horizon": 100,
    },
    characterization_methods=[
        {
            "category_name": "climate_change",
            "brightway_method": method,
            "metric": "CRF",
        },
    ],
)

In [10]:
lca_data_processor = lca_processor.LCADataProcessor(lca_config)

2025-06-06 15:43:12.244 | INFO     | optimex.lca_processor:_parse_demand:323 - Identified demand in system time range of %s for functional flows %s
2025-06-06 15:43:12.267 | INFO     | optimex.lca_processor:_construct_foreground_tensors:448 - Constructed foreground tensors.
2025-06-06 15:43:12.271 | INFO     | optimex.lca_processor:log_tensor_dimensions:443 - Technosphere shape: (2 processes, 10 flows, 29 years) with 148 total entries.
2025-06-06 15:43:12.272 | INFO     | optimex.lca_processor:log_tensor_dimensions:443 - Biosphere shape: (2 processes, 4 flows, 27 years) with 94 total entries.
2025-06-06 15:43:12.273 | INFO     | optimex.lca_processor:log_tensor_dimensions:443 - Production shape: (2 processes, 1 flows, 27 years) with 42 total entries.
2025-06-06 15:43:12.273 | INFO     | optimex.lca_processor:_calculate_inventory_of_db:486 - Calculating inventory for database: ei3.10-SSP2-RCP19-2020
2025-06-06 15:43:18.630 | INFO     | optimex.lca_processor:_calculate_inventory_of_db:50

ValueError: There are no flows to characterize. Please make sure your time horizon matches the             timing of emissions and make sure there are characterization functions for the flows             in the dynamic inventories.